# itkPolyDataToMeshFilter

This example serves to demonstrate use of `itkMeshToPolyDataFilter` and `itkPolyDataToMeshFilter` for conversion between `itk.Mesh` and `itk.PolyData` data structures. An `itk.Mesh` object stores geometric data as a collection of `CellInterface` objects defined on the IDs of an underlying `itk.PointSet`, while an `itk.PolyData` object stores geometric data as a collection of points and a list of point IDs segmented by cell. `itk.Mesh` supports standard mesh formats and is useful for I/O with `itk.meshread` and `itk.meshwrite`, while `itk.PolyData` is often useful for visualization with `vtk`.

In [1]:
import os
from urllib.request import urlretrieve

import itk
from itkwidgets import view

### Get Mesh Input

In [2]:
MESH_FILE = 'Input/cow.vtk'

# Download mesh
os.makedirs('Input', exist_ok=True)
if not os.path.exists(MESH_FILE):
    url = 'https://data.kitware.com/api/v1/file/5c43feba8d777f072b0cd3da/download'
    urlretrieve(url, MESH_FILE)

In [3]:
mesh_input = itk.meshread(MESH_FILE, itk.D)

In [4]:
print(mesh_input.GetNumberOfPoints())
print(mesh_input.GetNumberOfCells())

2903
3263


In [5]:
# Visualize points
for i in range(0,5):
    print(mesh_input.GetPoints().GetElement(i))

itkPointF3 ([3.71636, 2.34339, 0])
itkPointF3 ([4.12656, 0.642027, 0])
itkPointF3 ([3.45497, 2.16988, 0])
itkPointF3 ([3.92925, 0.411689, 0])
itkPointF3 ([3.24741, 2.07333, 0])


In [6]:
view(geometries=[mesh_input])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…

In [7]:
# Test preservation of point data
mesh_input.GetPointData().Reserve(1)
mesh_input.GetPointData().SetElement(0,500) #FIXME remove

### Convert to `itk.PolyData`

In [8]:
filter = itk.MeshToPolyDataFilter[type(mesh_input)].New(Input=mesh_input)

In [9]:
filter.Update()

In [10]:
poly_data = filter.GetOutput()

In [11]:
assert(type(poly_data) == itk.PolyData[itk.D])

In [12]:
# Visualize cells as collections of point IDs

i = 0
polygons = poly_data.GetPolygons()
while i < 1000:
    x = list()
    for j in range(0,polygons.ElementAt(i)):
        x.append(polygons.ElementAt(i + 1 + j))
    print(x)
    i += polygons.ElementAt(i) + 1

[250, 251, 210, 252]
[252, 210, 201, 253]
[253, 201, 254]
[254, 255, 256, 253]
[253, 256, 257, 252]
[252, 257, 258, 250]
[258, 257, 259, 260]
[260, 259, 261, 262]
[262, 261, 263]
[263, 261, 264, 265]
[265, 264, 266]
[266, 264, 267]
[267, 264, 268, 269]
[268, 264, 261, 259]
[259, 257, 256, 268]
[268, 256, 255, 269]
[255, 254, 270]
[269, 271, 272]
[272, 267, 269]
[272, 273, 274, 267]
[273, 272, 271]
[267, 274, 275]
[267, 275, 276, 266]
[266, 276, 277, 265]
[265, 277, 278]
[265, 278, 279, 263]
[263, 279, 280, 262]
[271, 281, 282, 273]
[273, 282, 283, 274]
[274, 283, 284, 275]
[284, 285, 276]
[276, 285, 286, 277]
[277, 286, 287, 278]
[278, 287, 288, 279]
[279, 288, 289, 280]
[281, 15, 16, 282]
[282, 16, 17, 283]
[283, 17, 18, 284]
[284, 18, 19, 285]
[285, 19, 20, 286]
[286, 20, 21, 287]
[287, 21, 22, 288]
[288, 22, 23, 289]
[290, 190, 291]
[291, 177, 290]
[290, 177, 292]
[292, 293, 290]
[290, 293, 191]
[254, 201, 192]
[192, 201, 200]
[200, 191, 192]
[254, 293, 292, 270]
[270, 292, 271]
[27

### Convert back to `itk.Mesh`

In [13]:
filter = itk.PolyDataToMeshFilter[type(poly_data)].New(Input=poly_data)

In [14]:
filter.Update()

In [15]:
mesh_output = filter.GetOutput()

In [16]:
assert(mesh_output.GetNumberOfPoints() == mesh_input.GetNumberOfPoints())
assert(mesh_output.GetNumberOfCells() == mesh_input.GetNumberOfCells())
for i in range(0,mesh_output.GetNumberOfPoints()):
    assert(mesh_output.GetPoint(i) == mesh_input.GetPoint(i))

In [17]:
view(geometries=[mesh_output])

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'numberOfComponents': 3, 'd…